In [26]:
import tushare as ts
import pymongo
import datetime
client = pymongo.MongoClient("127.0.0.1", 27017)

In [16]:
def getOneStockHistoryData(code, startDate):
    res = ts.get_k_data(code, start=startDate)
    db = client.Trade
    collection = db[code]
    count = 0
    for k, v in res.iterrows():
        date = datetime.datetime.strptime(str(v.date), "%Y-%m-%d")
        item = {
            "date": date,
            "open": v.open,
            "close": v.close,
            "high": v.high,
            "low": v.low,
            "volume": v.volume
        }

        if not collection.find_one({"date": date}):
            collection.insert_one(item)
            count = count + 1
    
    if count > 0:
        print("%s - %s rows updated" % (code, count))

In [17]:
def getHistoryData():
    collection = client.Common.Stock
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    stocks = collection.find({})
    count = 0
    for v in stocks:
        code = v["code"]
        if "kEndDate" in v:
            date = v["kEndDate"] + datetime.timedelta(days=-1)
        elif "foundTime" in v:
            date = v["foundTime"]
        else:
            date = datetime.datetime(2000, 1, 1)
        getOneStockHistoryData(code, date.strftime("%Y-%m-%d"))
        collection.update_one({"code": code}, {"$set": {"kEndDate": today}})
        count = count + 1
        if count % 500 == 0:
            print("%s processed" % (count))

In [27]:
def getTodayData():
    df = ts.get_today_all()
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    for _,v in df.iterrows():
        item = {
            "volume": v.volume / 100,
            "high": v.high,
            "low": v.low,
            "date": today,
            "close": v.trade,
            "open": v.open,
            "amount": v.amount,
            "last_close": v.settlement,
            "tradeValue": v.nmc,
        }
        code = v.code
        collection = client.Trade[code]
        collection.replace_one({"date": today}, item, upsert=True)
    print("Done")

In [30]:
getTodayData()

[Getting data:]############################################################Done
